In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
df = pd.read_csv('data/housing.csv')
df.head()



,Cena,Powierzchnia,Miasto,Liczba pokoi,Rok budowy
0,310000,40.80,Mikolow,1,1959
1,489000,40.84,Mikolow,1,2022
2,540000,60.00,Mikolow,3,1910
3,950000,125.40,Mikolow,4,2025
4,1250000,124.30,Mikolow,4,2025


In [9]:
df_num = df.select_dtypes(include=[np.number])
df_miasta = pd.get_dummies(df, columns=['Miasto'], drop_first=True)
df_num

,Cena,Powierzchnia,Liczba pokoi,Rok budowy
0,310000,40.80,1,1959
1,489000,40.84,1,2022
2,540000,60.00,3,1910
3,950000,125.40,4,2025
4,1250000,124.30,4,2025
...,...,...,...,...
75,430000,51.00,3,1957
76,598000,40.00,2,2024
77,279000,38.00,2,1970
78,370000,51.00,2,1980
